In [1]:
!apt-get update && apt-get install curl -y
!curl https://raw.githubusercontent.com/nautechsystems/nautilus_data/main/nautilus_data/hist_data_to_catalog.py | python -

'apt-get' is not recognized as an internal or external command,
operable program or batch file.


                         bid_price  ask_price  size
20200101 170000065                                 
2020-01-01 17:00:10.447    1.12120    1.12192     0
2020-01-01 17:00:10.498    1.12117    1.12161     0
2020-01-01 17:00:12.579    1.12120    1.12161     0
2020-01-01 17:00:12.630    1.12120    1.12172     0
2020-01-01 17:00:12.839    1.12120    1.12171     0
...                            ...        ...   ...
2020-01-31 16:59:53.963    1.10945    1.10961     0
2020-01-31 16:59:54.064    1.10945    1.10957     0
2020-01-31 16:59:54.711    1.10945    1.10960     0
2020-01-31 16:59:55.922    1.10945    1.10952     0
2020-01-31 16:59:56.198    1.10944    1.10952     0

[1637768 rows x 3 columns]
Preparing ticks...
Writing data to catalog...
Done


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100  2502  100  2502    0     0  10468      0 --:--:-- --:--:-- --:--:-- 10425
c:\Users\allay\.conda\envs\ntrader\Lib\site-packages\nautilus_trader\persistence\loaders.py:63: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


In [22]:
from nautilus_trader.backtest.node import BacktestDataConfig
from nautilus_trader.backtest.node import BacktestEngineConfig
from nautilus_trader.backtest.node import BacktestNode
from nautilus_trader.backtest.node import BacktestRunConfig
from nautilus_trader.backtest.node import BacktestVenueConfig
from nautilus_trader.config import ImportableStrategyConfig
from nautilus_trader.config import LoggingConfig
from nautilus_trader.model import Quantity
from nautilus_trader.model import QuoteTick
from nautilus_trader.persistence.catalog import ParquetDataCatalog
from nautilus_trader.backtest.results import BacktestResult
from nautilus_trader.backtest.engine import BacktestEngine
from nautilus_trader.model import Venue

In [3]:
from nautilus_trader.core.message import Event
from nautilus_trader.indicators.macd import MovingAverageConvergenceDivergence
from nautilus_trader.model import InstrumentId
from nautilus_trader.model import Position
from nautilus_trader.model.enums import OrderSide
from nautilus_trader.model.enums import PositionSide
from nautilus_trader.model.enums import PriceType
from nautilus_trader.model.events import PositionOpened
from nautilus_trader.trading.strategy import Strategy
from nautilus_trader.trading.strategy import StrategyConfig


class MACDConfig(StrategyConfig):
    instrument_id: InstrumentId
    fast_period: int = 12
    slow_period: int = 26
    trade_size: int = 1_000_000
    entry_threshold: float = 0.00010


class MACDStrategy(Strategy):
    def __init__(self, config: MACDConfig):
        super().__init__(config=config)
        # Our "trading signal"
        self.macd = MovingAverageConvergenceDivergence(
            fast_period=config.fast_period,
            slow_period=config.slow_period,
            price_type=PriceType.MID,
        )

        self.trade_size = Quantity.from_int(config.trade_size)

        # Convenience
        self.position: Position | None = None

    def on_start(self):
        self.subscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_stop(self):
        self.close_all_positions(self.config.instrument_id)
        self.unsubscribe_quote_ticks(instrument_id=self.config.instrument_id)

    def on_quote_tick(self, tick: QuoteTick):
        # You can register indicators to receive quote tick updates automatically,
        # here we manually update the indicator to demonstrate the flexibility available.
        self.macd.handle_quote_tick(tick)

        if not self.macd.initialized:
            return  # Wait for indicator to warm up

        # self._log.info(f"{self.macd.value=}:%5d")
        self.check_for_entry()
        self.check_for_exit()

    def on_event(self, event: Event):
        if isinstance(event, PositionOpened):
            self.position = self.cache.position(event.position_id)

    def check_for_entry(self):
        # If MACD line is above our entry threshold, we should be LONG
        if self.macd.value > self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.LONG:
                return  # Already LONG

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.BUY,
                quantity=self.trade_size,
            )
            self.submit_order(order)
        # If MACD line is below our entry threshold, we should be SHORT
        elif self.macd.value < -self.config.entry_threshold:
            if self.position and self.position.side == PositionSide.SHORT:
                return  # Already SHORT

            order = self.order_factory.market(
                instrument_id=self.config.instrument_id,
                order_side=OrderSide.SELL,
                quantity=self.trade_size,
            )
            self.submit_order(order)

    def check_for_exit(self):
        # If MACD line is above zero then exit if we are SHORT
        if self.macd.value >= 0.0:
            if self.position and self.position.side == PositionSide.SHORT:
                self.close_position(self.position)
        # If MACD line is below zero then exit if we are LONG
        else:
            if self.position and self.position.side == PositionSide.LONG:
                self.close_position(self.position)

    def on_dispose(self):
        pass  # Do nothing else

Configure Venue

In [4]:
venue = BacktestVenueConfig(
    name="SIM",
    oms_type="NETTING",
    account_type="MARGIN",
    base_currency="USD",
    starting_balances=["1_000_000 USD"],
)

Configure Data

In [11]:
catalog = ParquetDataCatalog(
    "C:\\Users\\allay\\OneDrive\\Documents\\Repositories\\Trading System\\Trading-ntrader\\catalog"
)
instruments = catalog.instruments()
instruments

[CurrencyPair(id=EUR/USD.SIM, raw_symbol=EUR/USD, asset_class=FX, instrument_class=SPOT, quote_currency=USD, is_inverse=False, price_precision=5, price_increment=0.00001, size_precision=0, size_increment=1, multiplier=1, lot_size=1000, margin_init=0.03, margin_maint=0.03, maker_fee=0.00002, taker_fee=0.00002, info=None)]

In [12]:
data = BacktestDataConfig(
    catalog_path=str(catalog.path),
    data_cls=QuoteTick,
    instrument_id=instruments[0].id,
    end_time="2020-01-10",
)

Configure Engine

In [13]:
# NautilusTrader currently exceeds the rate limit for Jupyter notebook logging (stdout output),
# this is why the `log_level` is set to "ERROR". If you lower this level to see
# more logging then the notebook will hang during cell execution. A fix is currently
# being investigated which involves either raising the configured rate limits for
# Jupyter, or throttling the log flushing from Nautilus.
# https://github.com/jupyterlab/jupyterlab/issues/12845
# https://github.com/deshaw/jupyterlab-limit-output
engine = BacktestEngineConfig(
    strategies=[
        ImportableStrategyConfig(
            strategy_path="__main__:MACDStrategy",
            config_path="__main__:MACDConfig",
            config={
                "instrument_id": instruments[0].id,
                "fast_period": 12,
                "slow_period": 26,
            },
        )
    ],
    logging=LoggingConfig(log_level="ERROR"),
)

Run backtest

In [14]:
config = BacktestRunConfig(
    engine=engine,
    venues=[venue],
    data=[data],
)

In [16]:
node = BacktestNode(configs=[config])

# Runs one or many configs synchronously
results: list[BacktestResult] = node.run()

Analyze results

In [18]:
engine: BacktestEngine = node.get_engine(config.id)

engine.trader.generate_order_fills_report()

,trader_id,strategy_id,instrument_id,venue_order_id,position_id,account_id,last_trade_id,type,side,quantity,...,order_list_id,linked_order_ids,parent_order_id,exec_algorithm_id,exec_algorithm_params,exec_spawn_id,tags,init_id,ts_init,ts_last
client_order_id,,,,,,,,,,,,,,,,,,,,,
O-20200103-100000-001-000-1,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-001,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-001,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,f52c475f-1caa-4593-9730-b6f86ef61a8a,2020-01-03 10:00:00.242000+00:00,2020-01-03 10:00:00.242000+00:00
O-20200103-100004-001-000-2,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-002,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-002,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,c305534c-7bd6-49e6-a772-e55bef5869f1,2020-01-03 10:00:04.136000+00:00,2020-01-03 10:00:04.136000+00:00
O-20200103-100006-001-000-3,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-003,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-003,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,ffb5e3d0-2d7d-4fc7-949e-d193ca5257de,2020-01-03 10:00:06.093000+00:00,2020-01-03 10:00:06.093000+00:00
O-20200103-100007-001-000-4,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-004,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-004,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,4d69ba48-9a33-4c56-bea7-943353d05a75,2020-01-03 10:00:07.548000+00:00,2020-01-03 10:00:07.548000+00:00
O-20200103-165230-001-000-5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-005,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-005,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,5bdae6a2-18ea-4550-8b53-139c412319c2,2020-01-03 16:52:30.901000+00:00,2020-01-03 16:52:30.901000+00:00
O-20200103-165246-001-000-6,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-006,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-006,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,dffc2c81-ead5-4164-9e65-76467e9eb4f5,2020-01-03 16:52:46.001000+00:00,2020-01-03 16:52:46.001000+00:00
O-20200105-170032-001-000-7,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-007,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-007,MARKET,BUY,1000000,...,None,None,None,None,None,None,None,3e90123c-3e54-411e-ade8-d528365cc51e,2020-01-05 17:00:32.758000+00:00,2020-01-05 17:00:32.758000+00:00
O-20200105-170530-001-000-8,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-008,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-008,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,1047993f-f5f9-49ca-94d3-99c4e8c1c5dd,2020-01-05 17:05:30.628000+00:00,2020-01-05 17:05:30.628000+00:00
O-20200108-031312-001-000-9,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-1-009,EUR/USD.SIM-MACDStrategy-000,SIM-001,SIM-1-009,MARKET,SELL,1000000,...,None,None,None,None,None,None,None,feb40937-c885-4de7-bf40-6fd4c4345a04,2020-01-08 03:13:12.834000+00:00,2020-01-08 03:13:12.834000+00:00


In [19]:
engine.trader.generate_positions_report()

,trader_id,strategy_id,instrument_id,account_id,opening_order_id,closing_order_id,entry,side,quantity,peak_qty,ts_init,ts_opened,ts_last,ts_closed,duration_ns,avg_px_open,avg_px_close,commissions,realized_return,realized_pnl
position_id,,,,,,,,,,,,,,,,,,,,
EUR/USD.SIM-MACDStrategy-000-c02af6a3-2b00-46c1-a6df-22fb3896ced0,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-100000-001-000-1,O-20200103-100004-001-000-2,BUY,FLAT,0,1000000,1578045600242000000,2020-01-03 10:00:00.242000+00:00,1578045604136000000,2020-01-03 10:00:04.136000+00:00,3894000000,1.11644,1.11635,[44.66 USD],-0.00008,-134.66 USD
EUR/USD.SIM-MACDStrategy-000-f3112ea7-6d5c-4ec7-a606-c980a842e82a,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-100006-001-000-3,O-20200103-100007-001-000-4,SELL,FLAT,0,1000000,1578045606093000000,2020-01-03 10:00:06.093000+00:00,1578045607548000000,2020-01-03 10:00:07.548000+00:00,1455000000,1.11604,1.11637,[44.65 USD],-0.00030,-374.65 USD
EUR/USD.SIM-MACDStrategy-000-75920140-0f9f-421d-9e70-2dd332466d40,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200103-165230-001-000-5,O-20200103-165246-001-000-6,SELL,FLAT,0,1000000,1578070350901000000,2020-01-03 16:52:30.901000+00:00,1578070366001000000,2020-01-03 16:52:46.001000+00:00,15100000000,1.11534,1.11543,[44.62 USD],-0.00008,-134.62 USD
EUR/USD.SIM-MACDStrategy-000-4d2e17b5-8dd4-45e0-b8c6-3b1eb94a3c72,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200105-170032-001-000-7,O-20200105-170530-001-000-8,BUY,FLAT,0,1000000,1578243632758000000,2020-01-05 17:00:32.758000+00:00,1578243930628000000,2020-01-05 17:05:30.628000+00:00,297870000000,1.11703,1.11652,[44.67 USD],-0.00046,-554.67 USD
EUR/USD.SIM-MACDStrategy-000-8c8f6d23-cf9a-477b-939e-53ca9c0a8862,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-031312-001-000-9,O-20200108-031316-001-000-10,SELL,FLAT,0,1000000,1578453192834000000,2020-01-08 03:13:12.834000+00:00,1578453196947000000,2020-01-08 03:13:16.947000+00:00,4113000000,1.11403,1.11360,[44.55 USD],0.00039,385.45 USD
EUR/USD.SIM-MACDStrategy-000-af2e4a67-a7e7-4383-a2b5-292443b6cc93,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-062139-001-000-11,O-20200108-062157-001-000-12,SELL,FLAT,0,1000000,1578464499588000000,2020-01-08 06:21:39.588000+00:00,1578464517525000000,2020-01-08 06:21:57.525000+00:00,17937000000,1.11231,1.11211,[44.49 USD],0.00018,155.51 USD
EUR/USD.SIM-MACDStrategy-000-f5fce3e7-bf3d-45c1-945f-fecbe3dcceb5,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-081501-001-000-13,O-20200108-081514-001-000-14,SELL,FLAT,0,1000000,1578471301502000000,2020-01-08 08:15:01.502000+00:00,1578471314869000000,2020-01-08 08:15:14.869000+00:00,13367000000,1.11121,1.11120,[44.44 USD],0.00001,-34.44 USD
EUR/USD.SIM-MACDStrategy-000,BACKTESTER-001,MACDStrategy-000,EUR/USD.SIM,SIM-001,O-20200108-172644-001-000-15,O-20200108-172650-001-000-16,BUY,FLAT,0,1000000,1578504404662000000,2020-01-08 17:26:44.662000+00:00,1578504410316000000,2020-01-08 17:26:50.316000+00:00,5654000000,1.11164,1.11121,[44.45 USD],-0.00039,-474.45 USD


In [23]:
engine.trader.generate_account_report(Venue("SIM"))

,total,locked,free,currency,account_id,account_type,base_currency,margins,reported,info
2020-01-01 17:00:10.447000+00:00,1000000.00,0.00,1000000.00,USD,SIM-001,MARGIN,USD,[],True,{}
2020-01-03 10:00:00.242000+00:00,999977.67,0.00,999977.67,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:04.136000+00:00,999865.34,33515.53,966349.81,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 10:00:06.093000+00:00,999843.02,0.00,999843.02,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 10:00:07.548000+00:00,999490.69,33503.52,965987.17,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-03 16:52:30.901000+00:00,999468.38,0.00,999468.38,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-03 16:52:46.001000+00:00,999356.07,33482.51,965873.56,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-05 17:00:32.758000+00:00,999333.73,0.00,999333.73,USD,SIM-001,MARGIN,USD,[],False,{}
2020-01-05 17:05:30.628000+00:00,998801.40,33533.24,965268.16,USD,SIM-001,MARGIN,USD,"[{'type': 'MarginBalance', 'initial': '0.00', ...",False,{}
2020-01-08 03:13:12.834000+00:00,998779.12,0.00,998779.12,USD,SIM-001,MARGIN,USD,[],False,{}
